In [1]:
import os
import gc
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm
from osgeo import gdal

In [9]:
# get the name, lon and lat of unique institutions, and summarise the ratio of valid data for each column (attribtue)
read_folder = r'C:\1-Data\higher education'
len_all = 0
dict_attribute = {}
df_institute_comb = None
for file in tqdm(os.listdir(read_folder)):
    df = pd.read_excel(read_folder + '\\' + file)
    len_all += len(df)
    df_institute_temp = df[df.columns[:5]]
    df_institute_comb = df_institute_temp if df_institute_comb is None else pd.concat([df_institute_comb, df_institute_temp])
    
    for column in df.columns[5:]:
        len_valid = df[column].apply(lambda x: str(x).replace('.', '', 1).isdigit()).sum()
        if column in dict_attribute.keys():
            dict_attribute[column] += len_valid
        else:
            dict_attribute[column] = len_valid
        

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [03:28<00:00, 18.95s/it]


In [114]:
# divide the number of valid data by the total number to get the ratio
df_attribute = pd.DataFrame([dict_attribute]).T
df_attribute.columns = ['ratio of valid data']
df_attribute['ratio of valid data'] = df_attribute['ratio of valid data']/len_all
df_attribute.to_excel(r'C:\1-Data\higher education indicators.xlsx')

In [29]:
# # save the df of unique institutes 
# df_institute_unique = df_institute_comb.drop_duplicates(subset='ETER ID')
# df_institute_unique.to_excel(r'C:\1-Data\higher education\unique_institutes0.xlsx')

In [3]:
# import list of unique institutes with missing lon and lat of institutes manually added
df_institute_unique = pd.read_excel(r'C:\1-Data\higher education\unique_institutes.xlsx', index_col=0)
df_institute_unique.rename(columns={'Geographic coordinates - latitude':'lat', 
                                    'Geographic coordinates - longitude':'lon'}, inplace=True)
# import the list of kept indicators
df_kept_indicators = pd.read_excel(r'C:\1-Data\higher education' + '\\' + 'higher education indicators.xlsx', sheet_name='keep', index_col=0)

In [ ]:
read_folder = r'C:\1-Data\higher education'
temp_folder = r'C:\2-Case studies\Higher Education Observatory\temp'
nuts_folder = r'C:\1-Data\NUTS'
years = [2003, 2006, 2010, 2013, 2016, 2021, 2024]
for year in tqdm(years):
    nuts = gpd.read_file(nuts_folder + '\\' + 'NUTS_RG_01M_' + str(year) + '_3035.shp')
    
    for ObsTime in tqdm(range(2011, 2022)):
        read_path = read_folder + '\\' + str(ObsTime) + '.xlsx'
        df_raw_institute = pd.read_excel(read_path, index_col=0)
        # append lon and lat data from df_institute_uniqe (because some lon and lat data are missing in raw data),
        # and convert raw data of each year to shp
        df_raw_institute = pd.merge(df_raw_institute, df_institute_unique[['ETER ID', 'lat', 'lon']], how='left', on='ETER ID')
        gdf_raw_institute = gpd.GeoDataFrame(df_raw_institute, geometry=gpd.points_from_xy(df_raw_institute.lon, df_raw_institute.lat),
                                    crs="EPSG:4326").to_crs('EPSG:3035')
        # spatial join nuts region and raw data of institutes
        gdf_joined = gpd.sjoin(gdf_raw_institute, nuts[['NUTS_ID', 'geometry']], how="left", predicate='within')
    
        # for each indicator, get the sum and export the df to a temp folder
        for indicator in df_kept_indicators.index:
            gdf_joined[indicator] = pd.to_numeric(gdf_joined[indicator], errors='coerce')
            grouped_sum = pd.DataFrame(gdf_joined.groupby('NUTS_ID')[indicator].sum()).reset_index()
            grouped_sum.columns = ['geo', 'ObsValue']
            grouped_sum['geo_source'] = 'NUTS' + str(year)
            grouped_sum['Indicator'] = indicator
            grouped_sum['freq'] = 'year'
            grouped_sum['ObsTime'] = ObsTime
            grouped_sum['Unit'] = 'See indicator'
            grouped_sum.to_csv(temp_folder + '\\' + indicator + '_' + str(ObsTime) + '_NUTS' + str(year) + '.csv')

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]